In [ ]:
# make sure we are working in module directory
repo_root = !git rev-parse --show-toplevel
module_path = repo_root[0] + "/backend/heatflask"
%cd $module_path

import sys
__package__ = "heatflask"
if ".." not in sys.path:
    sys.path.insert(0, "..")


# Make cells wider
from IPython.display import display, HTML
display(HTML("<style>.container { width:96% !important; }</style>"))

In [ ]:
%load Users.py

In [ ]:
# await DataAPIs.connect()
# await drop()
# await DataAPIs.disconnect()

In [ ]:
import logging
logging.basicConfig(level="DEBUG")

await DataAPIs.connect()

new_user_dict = {"username": "guy"}
await add_or_update(**new_user_dict)  # should give an error

new_user_dict = {**new_user_dict, "id": 222}
await add_or_update(**new_user_dict)

In [ ]:
await get(222)

In [ ]:
await add_or_update(_id=222, private=False)

await add_or_update(id=222, inc_access_count=True)

result = await add_or_update(id=222, update_ts=True)
user_info = await get(222)
result, user_info

In [ ]:
result = await delete(222)

In [ ]:
# import real user data from Strava
from . import Strava
print("Paste this URL into your browser and retrieve the code:\n", Strava.auth_url())
print("You will be redirected to an error page. copy the 'code' parameter from the url")

In [ ]:
# exchange code for token
CODE = "cda3bbe70722fa345770211c65f940f605c64598"
C = Strava.AsyncClient("admin")
auth_info = await C.update_access_token(code=CODE)
strava_athlete = auth_info.pop("athlete")
auth_info, strava_athlete

In [ ]:
mongo_doc(**strava_athlete, auth=auth_info)

In [ ]:
await add_or_update(
    update_last_login=True,
    update_index_access=True,
    inc_login_count=True,
    **strava_athlete,
    auth=auth_info
)

In [ ]:
# async_cursor = await get_all()
# userslist1 = [user async for user in async_cursor]  

async_cursor = await get_all()
userslist2 = await async_cursor.to_list(length=None)
userslist2

In [ ]:
await migrate()

In [ ]:
await DataAPIs.disconnect()

In [ ]:
await DataAPIs.connect()
all_users = [a async for a in dump(admin=True, output="csv")]
await DataAPIs.disconnect()
all_users

In [ ]:
await DataAPIs.connect()
stale_users = await triage()
await DataAPIs.disconnect()
stale_users